In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms

import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns

import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

In [2]:
# Setup google drive connection
from google.colab import drive
drive.mount('/content/drive')
import gspread
from google.auth import default
from google.colab import auth

# Autenticazione
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Mounted at /content/drive


In [3]:
# Setup git integration
%cd /content/drive/MyDrive
!git clone https://github.com/s348174/MLDL2024_project1.git
%cd

/content/drive/MyDrive
Cloning into 'MLDL2024_project1'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 80 (delta 25), reused 45 (delta 13), pack-reused 11 (from 2)
Receiving objects: 100% (80/80), 752.09 KiB | 8.09 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [ ]:
# Open zipfile
import zipfile
zip_path = '/content/drive/MyDrive/machine_learning_shared_2025/Cityscapes.zip'
with zipfile.ZipFile(zip_path) as z:
  # Mostra i file contenuti
  print("Contenuto dello ZIP:", z.namelist())
  with TemporaryDirectory() as tmpdir:
    z.extractall(tmpdir)
    image_folder_path=tmpdir+"/Cityscapes/Cityspaces/images/train"
    train_data = datasets.ImageFolder(
        root=image_folder_path,
        transform=ToTensor()
    )
    test_folder_path=tmpdir+"/Cityscapes/Cityspaces/images/test"
    test_data = datasets.ImageFolder(
        root=image_folder_path,
        transform=ToTensor()
    )

Contenuto dello ZIP: ['Cityscapes/Cityspaces/', 'Cityscapes/Cityspaces/gtFine/', 'Cityscapes/Cityspaces/gtFine/train/', 'Cityscapes/Cityspaces/gtFine/train/hanover/', 'Cityscapes/Cityspaces/gtFine/train/hanover/hanover_000000_000164_gtFine_color.png', 'Cityscapes/Cityspaces/gtFine/train/hanover/hanover_000000_000164_gtFine_labelTrainIds.png', 'Cityscapes/Cityspaces/gtFine/train/hanover/hanover_000000_000381_gtFine_color.png', 'Cityscapes/Cityspaces/gtFine/train/hanover/hanover_000000_000381_gtFine_labelTrainIds.png', 'Cityscapes/Cityspaces/gtFine/train/hanover/hanover_000000_000712_gtFine_color.png', 'Cityscapes/Cityspaces/gtFine/train/hanover/hanover_000000_000712_gtFine_labelTrainIds.png', 'Cityscapes/Cityspaces/gtFine/train/hanover/hanover_000000_001173_gtFine_color.png', 'Cityscapes/Cityspaces/gtFine/train/hanover/hanover_000000_001173_gtFine_labelTrainIds.png', 'Cityscapes/Cityspaces/gtFine/train/hanover/hanover_000000_001620_gtFine_color.png', 'Cityscapes/Cityspaces/gtFine/train/